# ISE 244 - Tabular Data Preprocessing

## Aditya Sahu

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [3]:
df=pd.read_csv('/content/drive/MyDrive/SJSUWork/ISE244/training_wids2024C1.csv')
df.head()

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,475714,NaN,MEDICAID,CA,924,84,F,NaN,C50919,Malignant neoplasm of unsp site of unspecified...,...,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,349367,White,COMMERCIAL,CA,928,62,F,28.49,C50411,Malig neoplm of upper-outer quadrant of right ...,...,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,138632,White,COMMERCIAL,TX,760,43,F,38.09,C50112,Malignant neoplasm of central portion of left ...,...,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,617843,White,COMMERCIAL,CA,926,45,F,NaN,C50212,Malig neoplasm of upper-inner quadrant of left...,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,0
4,817482,NaN,COMMERCIAL,ID,836,55,F,NaN,1749,"Malignant neoplasm of breast (female), unspeci...",...,15.276000,11.224000,1.946000,26.170213,12.088000,13.106000,41.356058,4.110749,11.722197,0


In [4]:
df.describe()

,patient_id,patient_zip3,patient_age,bmi,population,density,age_median,age_under_10,age_10_to_19,age_20s,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
count,12906.000000,12906.000000,12906.000000,3941.000000,12905.000000,12905.000000,12905.000000,12905.000000,12905.000000,12905.000000,...,12905.000000,12902.000000,12902.000000,12905.000000,12905.000000,12905.000000,12877.000000,12877.000000,12877.000000,12906.000000
mean,547381.196033,573.754300,59.183326,28.984539,20744.441237,1581.950419,40.502259,11.122784,12.945265,13.290376,...,13.335299,13.406950,4.474956,27.978387,8.575284,7.083376,39.822352,7.475221,16.098988,0.624516
std,260404.959974,275.447534,13.335216,5.696906,13886.903756,2966.305306,4.036963,1.512376,1.923974,3.354103,...,3.690949,5.222495,4.837085,5.083939,4.203482,3.109022,3.559492,1.516499,5.842501,0.484266
min,100063.000000,101.000000,18.000000,14.000000,635.545455,0.916667,20.600000,0.000000,6.314286,5.925000,...,4.600000,3.433333,0.000000,12.460784,2.440000,1.200000,30.939316,2.636008,2.760371,0.000000
25%,321517.000000,331.000000,50.000000,24.660000,9463.896552,171.857143,37.129825,10.160000,11.741176,11.013415,...,10.270492,9.663333,0.994444,24.933333,5.618750,4.929688,37.698880,6.651215,11.280694,0.000000
50%,543522.000000,554.000000,59.000000,28.190000,19154.190480,700.337500,40.639344,11.039216,12.923944,12.538095,...,12.884000,12.177778,2.747222,27.788235,7.465714,6.847059,39.108249,7.686577,15.589148,1.000000
75%,772671.750000,846.000000,67.000000,32.920000,30021.278690,1666.515385,42.934783,12.190000,14.019767,14.971053,...,15.555405,16.635556,5.976000,30.709375,10.617442,8.620000,41.136513,8.276922,20.801880,1.000000
max,999896.000000,999.000000,91.000000,85.000000,71374.131580,21172.000000,54.570000,17.675000,35.300000,62.100000,...,35.155556,38.347826,26.755000,48.020000,27.566102,25.200000,52.237210,11.169408,31.504775,1.000000


In [5]:
print("Shape",df.shape)
sum(df.isnull().sum())

Shape (12906, 83)


43292

In [6]:
# Print missing values in ascending order
print("\nMissing values in the dataset (in ascending order):")
missing_values = df.isnull().sum().sort_values(ascending=True)
print(missing_values)


Missing values in the dataset (in ascending order):
patient_id                                   0
metastatic_cancer_diagnosis_code             0
breast_cancer_diagnosis_desc                 0
breast_cancer_diagnosis_code                 0
patient_gender                               0
                                         ...  
payer_type                                1803
patient_race                              6385
bmi                                       8965
metastatic_first_novel_treatment         12882
metastatic_first_novel_treatment_type    12882
Length: 83, dtype: int64


We will drop the last 4 columns which have more than half entries empty.

In [7]:
columns_to_drop = ['patient_race', 'bmi', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']
df = df.drop(columns_to_drop, axis=1)
df.head()

,patient_id,payer_type,patient_state,patient_zip3,patient_age,patient_gender,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,Region,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,475714,MEDICAID,CA,924,84,F,C50919,Malignant neoplasm of unsp site of unspecified...,C7989,West,...,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,349367,COMMERCIAL,CA,928,62,F,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,West,...,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,138632,COMMERCIAL,TX,760,43,F,C50112,Malignant neoplasm of central portion of left ...,C773,South,...,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,617843,COMMERCIAL,CA,926,45,F,C50212,Malig neoplasm of upper-inner quadrant of left...,C773,West,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,0
4,817482,COMMERCIAL,ID,836,55,F,1749,"Malignant neoplasm of breast (female), unspeci...",C773,West,...,15.276000,11.224000,1.946000,26.170213,12.088000,13.106000,41.356058,4.110749,11.722197,0


In [8]:
print("Data types of columns in the DataFrame:")
print(df.dtypes)

Data types of columns in the DataFrame:
patient_id          int64
payer_type         object
patient_state      object
patient_zip3        int64
patient_age         int64
                   ...   
veteran           float64
Ozone             float64
PM25              float64
N02               float64
DiagPeriodL90D      int64
Length: 79, dtype: object


Creating a Pipeline for Streamlining the process for:
1. Label encoding of Target class
2. Imputing the missing data
3. Scaling and normalizing the numeric and categorical columns

In [9]:
# Separate features (X) and target variable (y)
X = df.drop(columns=['DiagPeriodL90D'])
y = df['DiagPeriodL90D']

# Convert target variable to binary values (0 and 1)
label_encoder = LabelEncoder()
y_label_encoded = label_encoder.fit_transform(y)
# Specify numeric and categorical columns
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

# Create transformers for numeric and categorical columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers into a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, selector(dtype_exclude="object")),
        ('cat', categorical_transformer, selector(dtype_include="object"))
    ]
)

# Create the final pipeline with the preprocessor and any additional steps (e.g., modeling)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # Add additional steps if needed
])

# Fit and transform the data through the pipeline
X_transformed = pipeline.fit_transform(X)
X_train, X_val, y_train, y_val = train_test_split(X_transformed, y_label_encoded, test_size=0.2, random_state=42)


# Model 1: RandomForestClassifier

In [10]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_val)

# Evaluate the model
accuracy_rf = accuracy_score(y_val, y_pred_rf)
print(f"Random Forest Classifier Accuracy: {accuracy_rf * 100:.2f}%")

Random Forest Classifier Accuracy: 77.46%


# Model 2: XGBoost Classifier

In [11]:
import xgboost as xgb

# Define XGBoost parameters
params = {
    #'objective': 'binary:logistic',  # for binary classification
    'eval_metric': 'logloss',  # log likelihood loss for binary classification
    'max_depth': 6,  # maximum depth of the tree
    'min_child_weight': 1,  # minimum sum of instance weight (hessian) needed in a child
    'subsample': 0.8,  # subsample ratio of the training instances
    'colsample_bytree': 0.8,  # subsample ratio of columns when constructing each tree
    'learning_rate': 0.1,  # step size shrinkage to prevent overfitting
    'n_estimators': 100,  # number of boosting rounds
    'seed': 42
}

# Initialize XGBoost model
model_xgb = xgb.XGBClassifier(**params)

# Train the model
model_xgb.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = model_xgb.predict(X_val)

# Evaluate the model
accuracy_xgb = accuracy_score(y_val, y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy_xgb * 100:.2f}%")

XGBoost Accuracy: 81.14%


# Model 3: LightBGM

In [12]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# Set hyperparameters for the LightGBM model
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the LightGBM model
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Make predictions on the test set
y_pred_lgb = bst.predict(X_val, num_iteration=bst.best_iteration)
y_pred_binary_lbg = np.round(y_pred_lgb)

# Evaluate the model
accuracy_lgb = accuracy_score(y_val, y_pred_binary_lbg)
print(f"LightBGM Accuracy: {accuracy_lgb * 100:.2f}%")

[LightGBM] [Info] Number of positive: 6443, number of negative: 3881
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008192 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17675
[LightGBM] [Info] Number of data points in the train set: 10324, number of used features: 199
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.624080 -> initscore=0.506901
[LightGBM] [Info] Start training from score 0.506901
LightBGM Accuracy: 81.33%


Final Model Comparision

1. Random Forest:

Accuracy: 77.46%
Random Forest performed reasonably well but fell slightly short compared to XGBoost.

2. XGBoost:

Accuracy: 81.14%
XGBoost demonstrated solid overall performance in classifying the target variable.


3. LightGBM:

Accuracy: 81.33%
LightGBM outperformed both XGBoost and Random Forest, showcasing the effectiveness of its gradient boosting algorithm on your dataset.

# Bonus: Kaggle Submission
Kaggle notebook can be found here with XGBoost as main model considered.
https://www.kaggle.com/code/adizz2407/xgboost-wids

In [13]:
df_test=pd.read_csv('/content/drive/MyDrive/SJSUWork/ISE244/test.csv')
df_sub=pd.read_csv('/content/drive/MyDrive/SJSUWork/ISE244/sample_submission.csv')
df_test.head()

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,...,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
0,573710,White,MEDICAID,IN,467,54,F,NaN,C50412,Malig neoplasm of upper-outer quadrant of left...,...,3.564516,13.996774,7.985484,0.969355,24.955357,10.838710,8.080645,38.724876,7.947165,11.157161
1,593679,NaN,COMMERCIAL,FL,337,52,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,...,10.261538,16.020513,13.602564,2.836842,23.952632,10.579487,9.302564,36.918257,7.838973,13.599985
2,184532,Hispanic,MEDICAID,CA,917,61,F,NaN,C50911,Malignant neoplasm of unsp site of right femal...,...,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
3,447383,Hispanic,MEDICARE ADVANTAGE,CA,917,64,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,...,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
4,687972,Black,NaN,CA,900,40,F,23.0,C50412,Malig neoplasm of upper-outer quadrant of left...,...,45.526154,11.901538,20.760000,14.737500,30.709375,10.341538,3.030769,41.186992,11.166898,21.644261


In [14]:
df_test = df_test.drop(columns_to_drop, axis=1)
print("Shape",df_test.shape)
sum(df.isnull().sum())

Shape (5792, 78)


2178

In [15]:
df_sub.head()

,patient_id,DiagPeriodL90D
0,573710,0.5
1,593679,0.5
2,184532,0.5
3,447383,0.5
4,687972,0.5


In [16]:
X_test_transformed = pipeline.transform(df_test)

# Make predictions using the XGBoost model
y_test_pred_xgb = model_xgb.predict(X_test_transformed)


# Insert the predicted values into the 'DiagPeriodL90D' column
df_test['DiagPeriodL90D'] = y_test_pred_xgb

# Save the DataFrame to a CSV file
#submission_df.to_csv('submission.csv', index=False)
df_test.head()

,patient_id,payer_type,patient_state,patient_zip3,patient_age,patient_gender,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,Region,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,573710,MEDICAID,IN,467,54,F,C50412,Malig neoplasm of upper-outer quadrant of left...,C773,Midwest,...,13.996774,7.985484,0.969355,24.955357,10.838710,8.080645,38.724876,7.947165,11.157161,1
1,593679,COMMERCIAL,FL,337,52,F,C50912,Malignant neoplasm of unspecified site of left...,C787,South,...,16.020513,13.602564,2.836842,23.952632,10.579487,9.302564,36.918257,7.838973,13.599985,1
2,184532,MEDICAID,CA,917,61,F,C50911,Malignant neoplasm of unsp site of right femal...,C773,West,...,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231,1
3,447383,MEDICARE ADVANTAGE,CA,917,64,F,C50912,Malignant neoplasm of unspecified site of left...,C779,West,...,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231,1
4,687972,NaN,CA,900,40,F,C50412,Malig neoplasm of upper-outer quadrant of left...,C779,West,...,11.901538,20.760000,14.737500,30.709375,10.341538,3.030769,41.186992,11.166898,21.644261,1


In [17]:
df_sub.shape

(5792, 2)

In [18]:
# Merge df_sub and df_test based on 'patient_id'
df_merged = pd.merge(df_sub, df_test[['patient_id', 'DiagPeriodL90D']], on='patient_id', how='left')
print(df_sub.shape)
df_merged.head()


(5792, 2)


,patient_id,DiagPeriodL90D_x,DiagPeriodL90D_y
0,573710,0.5,1
1,593679,0.5,1
2,184532,0.5,1
3,447383,0.5,1
4,687972,0.5,1


In [19]:
df_merged.describe()

,patient_id,DiagPeriodL90D_x,DiagPeriodL90D_y
count,5792.000000,5792.0,5792.000000
mean,549946.787983,0.5,0.754144
std,260988.833446,0.0,0.430631
min,100266.000000,0.5,0.000000
25%,320284.500000,0.5,1.000000
50%,553042.000000,0.5,1.000000
75%,778552.250000,0.5,1.000000
max,999890.000000,0.5,1.000000


In [20]:
# Replace the values in 'DiagPeriodL90D' with the predicted values
df_merged['DiagPeriodL90D'] = df_merged['DiagPeriodL90D_y'].combine_first(df_merged['DiagPeriodL90D_x'])

# Drop the redundant columns
df_merged.drop(['DiagPeriodL90D_x', 'DiagPeriodL90D_y'], axis=1, inplace=True)

# Display the merged DataFrame with updated 'DiagPeriodL90D'
print(df_merged)

      patient_id  DiagPeriodL90D
0         573710               1
1         593679               1
2         184532               1
3         447383               1
4         687972               1
...          ...             ...
5787      977076               1
5788      922960               1
5789      759690               1
5790      911717               0
5791      327597               1

[5792 rows x 2 columns]


In [21]:
df_merged.to_csv('submission.csv', index=False)
